In [1]:
import sys
sys.path.append("models/")

%load_ext autoreload
%autoreload 2
from collections import OrderedDict
import os
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import pickle as pkl
import numpy as np

import itertools
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate, GroupKFold
import statsmodels.api as sm


from dataloader import SurveyDataset, load_aggregate_travel_behavior, load_demo_v1
from M1_util_train_test import load_model, test
import linear_reg
import mnl
from setup import out_dir, data_dir, image_dir, model_dir, proj_dir


In [2]:
data_version = '1571'

model_type = 'AE'
sampling = 's'

zoomlevel = 'zoom15'

variable_names = ['active','auto','mas','pt', 'trpgen']

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']


# Load Model Embeddings

In [3]:
with open(proj_dir+"latent_space/2023-07-04T17-20-36_sae_kl_f32_eplast.pkl", "rb") as f:
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)
    sup_true_list = pkl.load(f)
    sup_list = pkl.load(f)

In [4]:
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
aggregate_embeddings = []
for i in unique_ct:
#     aggregate_embeddings.append(np.mean(sup_list[ct == i], axis=0))
    aggregate_embeddings.append(np.mean(encoder_output[ct == i], axis=0))
aggregate_embeddings = np.array(aggregate_embeddings)

x = aggregate_embeddings

x = x.reshape(x.shape[0],-1)

In [5]:
print(x.shape)

(1588, 10)


# Load Trip Behavior

In [6]:
file = "MyDailyTravel/origin_trip_behavior.csv"
df_pivot = load_aggregate_travel_behavior(file, data_version)
y_ct = df_pivot['geoid'].to_list()
y = df_pivot[variable_names].to_numpy()[:,:4]

groups = df_pivot['train_test']
group_split = GroupKFold(n_splits=5)

In [7]:
if len(x) != len(y):
    x_mask = [True if c in y_ct else False for c in unique_ct]
    x = x[x_mask,:]
    unique_ct = list(np.array(unique_ct)[x_mask])
    y_mask = [True if c in unique_ct else False for c in y_ct]
    y = y[y_mask,:]
    y_ct = list(np.array(y_ct)[y_mask])

x = x[[y_ct.index(val) for val in unique_ct],:]
unique_ct = list(np.array(unique_ct)[np.array([y_ct.index(val) for val in unique_ct])])
for xc,yc in zip(unique_ct, y_ct):
    assert xc == yc

In [8]:
print(len(y))

1571


# 1. Linear Regression

### 1.1 Auto Share

In [9]:
y_index = 1

In [10]:
# Lasso
for a in (1e-3)*np.array([0]):#3,4,5,6,7,8,9,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 0.00e+00 	 Train score: 0.5790 	 Cross val score: 0.5664 	 Nonzero coef: 10


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.430e+01, tolerance: 6.768e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in skl

### 1.2 PT

In [11]:
y_index = 3

In [12]:
# Lasso
for a in (1e-3)*np.array([0]):#3,4,5,6,7,8,9]):
# for a in (1e-4)*np.array([10,15,20,25,30,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 0.00e+00 	 Train score: 0.4621 	 Cross val score: 0.4427 	 Nonzero coef: 10


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.148e+00, tolerance: 1.168e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in skl

### 1.3 Active

In [13]:
y_index = 0

In [14]:
# Lasso
for a in (1e-3)*np.array([0]):#3,4,5,6,7,8,9,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 0.00e+00 	 Train score: 0.4721 	 Cross val score: 0.4568 	 Nonzero coef: 10


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.933e+00, tolerance: 3.383e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in skl

# 2. MNL for Mode Share

In [17]:
from util_aggregate_models import mnl_torch

# dataloader and model definition

lr_list = [5e-3]
wd_list = [1e-4, 1e-5,0]

results = {}
for i in range(5):
    
    train_filter = groups != i
    test_filter = groups == i
    x_train = x[train_filter]
    y_train = y[train_filter]
    x_test = x[test_filter]
    y_test = y[test_filter]
    sst_train = np.sum(np.power(y_train - np.mean(y_train, axis=0), 2), axis=0)
    sst_test = np.sum(np.power(y_test - np.mean(y_test, axis=0), 2), axis=0)
    
    trainset = SurveyDataset(torch.tensor(x_train,  dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

    testset = SurveyDataset(torch.tensor(x_test, dtype=torch.float), torch.tensor(y_test, dtype=torch.float))
    testloader = DataLoader(testset, batch_size=len(testset), shuffle=False)

    ret_dict = mnl_torch(trainloader, testloader, x_train.shape[-1], sst_train, sst_test, lr_list=lr_list, wd_list=wd_list)
    
    results[i] = ret_dict
    

[lr: 5.00e-03, wd: 1.00e-04]
Early stopping at epoch 470
[epoch: 465] Train KL loss: 0.139 Train R2 score: 0.477 0.590 -0.057 0.429 
[epoch: 465] Test KL loss: 0.145 Test R2 score: 0.474 0.581 -0.038 0.437 

[lr: 5.00e-03, wd: 1.00e-05]
[epoch: 995] Train KL loss: 0.145 Train R2 score: 0.469 0.558 -0.019 0.312 
[epoch: 995] Test KL loss: 0.150 Test R2 score: 0.443 0.555 -0.007 0.359 

[lr: 5.00e-03, wd: 0.00e+00]
Early stopping at epoch 855
[epoch: 850] Train KL loss: 0.145 Train R2 score: 0.447 0.560 -0.156 0.452 
[epoch: 850] Test KL loss: 0.151 Test R2 score: 0.419 0.541 -0.077 0.483 

[lr: 5.00e-03, wd: 1.00e-04]
Early stopping at epoch 615
[epoch: 610] Train KL loss: 0.138 Train R2 score: 0.482 0.592 -0.073 0.500 
[epoch: 610] Test KL loss: 0.152 Test R2 score: 0.428 0.577 -0.063 0.423 

[lr: 5.00e-03, wd: 1.00e-05]
Early stopping at epoch 725
[epoch: 720] Train KL loss: 0.141 Train R2 score: 0.485 0.581 -0.164 0.487 
[epoch: 720] Test KL loss: 0.156 Test R2 score: 0.429 0.561 -0.

In [18]:
df = []

for (lr, wd) in itertools.product(lr_list, wd_list):

    new = []
    
    for i in range(5):
        new2 = []
        for k,v in results[i][(lr,wd)].items():
            new2.append(results[i][(lr,wd)][k]) 
        new.append(new2)
        
    new = np.array(new) 
    
    df.append([lr] + [wd] + list(np.mean(new, axis=0)))

pd.DataFrame(np.array(df), columns = ['learning_rate','weight_decay','train_kl_loss','test_kl_loss','train_r2_auto','train_r2_active','train_r2_pt',
                                     'test_r2_auto','test_r2_active','test_r2_pt']).sort_values(by='test_kl_loss').round(4)

,learning_rate,weight_decay,train_kl_loss,test_kl_loss,train_r2_auto,train_r2_active,train_r2_pt,test_r2_auto,test_r2_active,test_r2_pt
2,0.005,0.0000,0.1417,0.1442,0.5773,0.4713,0.4361,0.5678,0.4527,0.4265
1,0.005,0.0000,0.1508,0.1537,0.5464,0.4140,0.4168,0.5280,0.3830,0.4239
0,0.005,0.0001,0.2029,0.2059,0.3231,0.3216,0.3498,0.2949,0.2806,0.3222
